## Detecção de Bordas
O objetivo dessa tarefa é implementar dois algoritmos. Depois que os algoritmos forem executados, as bordas serão representadas por pixels pretos enquanto os outros serão brancos. 

In [17]:
# Importando as dependências para trabalhar com imagens
import numpy as np
from PIL import Image

### Filtros
Para aplicar o filtro, precisamos de uma função de aplicação. No nosso caso, podemos imaginar nossos filtros como matrizes 3x3. Abaixo, segue os exemplos dos filtros necessários.
$$\text{1. Filtro Gaussiano:}\quad\frac{1}{16}\begin{bmatrix}
1 & 2 & 1 \\
2 & 4 & 2 \\
1 & 2 & 1 \\
\end{bmatrix}$$
$$\text{2. Filtro de Sobel em $x$:}\quad\begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1 \\
\end{bmatrix}$$
$$\text{3. Filtro de Sobel em $y$:}\quad\begin{bmatrix}
-1 & -2 & -1 \\
0 & 0 & 0 \\
1 & 2 & 1 \\
\end{bmatrix}$$
$$\text{4. Filtro Laplaciano:}\quad\begin{bmatrix}
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0 \\
\end{bmatrix}$$

In [18]:
gaussian = (1.0/16.0) * np.array([
    [1.0, 2.0, 1.0],
    [2.0, 4.0, 2.0],
    [1.0, 2.0, 1.0]
])

sobel_x = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])

sobel_y = np.array([
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
])

laplacian = np.array([
    [0, 1, 0],
    [1, -4, 1],
    [0, 1, 0]
])

### Aplicando os Filtros
Para aplicar os filtros, precisamos definir uma função que percorre a imagem, aplicando o filtro como uma máscara, para cada pixel, atribuindo o valor adquirido pós aplicação a uma nova matriz. A matriz resultante será a imagem final.

In [20]:
def apply_filter(image, filter):
  width = image.shape[0]
  height = image.shape[1]
  # Cria uma imagem nova, mas com uma borda "expandida" com zeros, para não dar problema.
  image_pad = np.pad(image, 1, mode = 'constant', constant_values = 0) 
  output = np.zeros(image.shape)
  for i in range(width):
    for j in range(height):
      mask = np.array([
        [image_pad[i-1][j-1], image_pad[i-1][j], image_pad[i-1][j+1]],
        [image_pad[i][j-1], image_pad[i][j], image_pad[i][j+1]],
        [image_pad[i+1][j-1], image_pad[i+1][j], image_pad[i+1][j+1]]
        ])
      output[i][j] = np.dot(filter[0], mask[0]) + np.dot(filter[1], mask[1]) + np.dot(filter[2], mask[2])
      
  return output

### Desenvolvendo os Algoritmos
Agora, temos todas as ferramentas prontas para aplicar os algoritmos de fato.

Para simplificação, Pi significa: *matriz do passo i*.

In [14]:
# Carregando as imagens
image_path = 'input/teste.webp'
input_img = np.array(Image.open(image_path).convert('L')) / 255

# O primeiro passo, para os dois algoritmos, é aplicar o filtro gaussiano:
P1 = apply_filter(input_img, gaussian)

#### Algoritmo 1

In [15]:
threshold = 0.5
A = apply_filter(P1, sobel_x)
B = apply_filter(P1, sobel_y)
C = np.sqrt(np.power(A, 2) + np.power(B, 2))
D = np.where(C < threshold, 0, 1)
output_img = Image.fromarray((D * 255).astype(np.uint8), mode='L')
output_img.save('output/alg1.png')

#### Algoritmo 2

In [21]:
tolerance = 0.04
A = apply_filter(P1, laplacian)
B = np.where(A <= tolerance, 0, 1)
output_img = Image.fromarray((B * 255).astype(np.uint8), mode='L')
output_img.save('output/alg2.png')